In [2]:
import glob
import json
from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim
from gensim.models.phrases import Phrases
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata
import datetime
import os
import time
import sys
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
print("getting data")
# get random samples from each party of similar sizes
datafiles = glob.glob("/data/shruti/ONR/big_data/Twitter/*.csv")
plist = ['_BJP.csv', '_INC.csv', '_AAP.csv']
pcount = defaultdict(list)
for d in datafiles:
    for p in plist:
        if p in d:
            pcount[p].append(d)
                
final_files = pcount
#final_files = defaultdict(list)
# for p in pcount:
#     try:
#         final_files[p] = sample(pcount[p], 1000)
#     except:
#         final_files[p] = sample(pcount[p], 430)
    
    
#read randomly sampled data into a dataframe
rowlist = []
for p in final_files.keys():
    party = p.replace("_", "").replace(".csv", "")
    for d in final_files[p]:
        with open(d, "r") as jsonfile:
            for line in jsonfile:
                job= json.loads(line)
                if "text" in job:
                    text = job['text']
                else:
                    text = None
                if "screen_name" in job:
                    sn = job['screen_name']
                else:
                    sn = None
                    
                if "lang" in job:
                    language = job['lang']
                else:
                    language = None
                    
                row = [sn, text, party, language]
                rowlist.append(row)

                
frame = pd.DataFrame(rowlist, columns=['screen_name','text','party', 'language'])
frame = frame.dropna(subset=['text'])

In [ ]:
frame = pd.read_csv("/data/shruti/ONR/small_data/Twitter_text/all_text.csv", header=0, lineterminator='\n',
                    error_bad_lines=False, encoding_errors='ignore')
frame.head()

In [ ]:
len(frame)

In [ ]:
len(frame)

In [ ]:
#frame.to_csv("/data/shruti/ONR/small_data/Twitter_text/all_text.csv")

In [ ]:
frame = frame.dropna(subset=['text'])

In [ ]:
frame = frame.dropna(subset=['screen_name'])

In [ ]:
len(frame)

In [ ]:
frame = frame.drop_duplicates(subset=['screen_name', 'text'])

In [ ]:
len(frame)

In [ ]:
def remove_retweets(text):
    if "RT " in text:
    #if text.startswith("RT "):
        return 1

In [ ]:
frame['is_retweet'] = frame['text'].parallel_apply(lambda x: remove_retweets(x))

In [ ]:
frame.head()

In [ ]:
data = frame.loc[frame['is_retweet']!=1.0]
len(data)

In [ ]:
len(data.drop_duplicates(subset=['screen_name','text']))

In [ ]:
data.sample(5)

In [ ]:
8913413 - 8666119

In [ ]:
data.dropna(subset=['screen_name']).head()

In [ ]:
len(data.drop_duplicates(subset=['text']))

In [ ]:
5594412 - 5464828

In [ ]:
data['is_duplicated'] = data.duplicated(subset=['text'], keep=False)
data.head()

In [ ]:
dup_data = data.loc[data['is_duplicated']==True]
dup_data.head()

In [ ]:
8913413 - 3822026

In [ ]:
unique_texts = list(set(dup_data['text'].tolist()))

In [ ]:
len(unique_texts)

In [ ]:
def assign_unique_group(text):
    uid = unique_texts.index(text)
    return uid

In [ ]:
dup_data['text_id'] = dup_data['text'].parallel_apply(lambda x: assign_unique_group(x))

In [ ]:
dup_data.to_csv("/data/shruti/ONR/small_data/Twitter_text/non_retweet_duplicate_text.csv")

In [4]:
dup_data = pd.read_csv("/data/shruti/ONR/small_data/Twitter_text/non_retweet_duplicate_text.csv", header=0, 
                       lineterminator='\n',
                    error_bad_lines=False, encoding_errors='ignore')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
dup_data.sort_values(by=['text_id'])

,Unnamed: 0,Unnamed: 0.1,screen_name,text,party,language,is_retweet,is_duplicated,text_id
41791,2940236,2940236,sachinkhurai,@bhupendrasingho ॐ शांति,BJP,hi,NaN,True,0
42350,2982692,2982692,BjpYova_Rahul,@bhupendrasingho ॐ शांति,BJP,hi,NaN,True,0
39914,2792355,2792355,jivaramchoudhar,श्रीगंगानगर में भाजपा SC मोर्चा के प्रदेशाध्यक...,BJP,hi,NaN,True,1
92504,6555844,6555844,VasundharaBJP,श्रीगंगानगर में भाजपा SC मोर्चा के प्रदेशाध्यक...,BJP,hi,NaN,True,1
162834,11755060,11755060,syedmusyeeb,ದೇಶಭಕ್ತರೆಂದು ಕರೆದುಕೊಳ್ಳುವ RSSಗೆ ತ್ರಿವರ್ಣ ಧ್ವಜವ...,INC,kn,NaN,True,2
...,...,...,...,...,...,...,...,...,...
97961,6876855,6876855,drmanojrajoria,जब फौज के वीर जवानों के पास आधुनिक हथियार होते...,BJP,hi,NaN,True,65724
23948,1495956,1495956,BJP4Rajasthan,जब फौज के वीर जवानों के पास आधुनिक हथियार होते...,BJP,hi,NaN,True,65724
63159,4443167,4443167,BJP4India,जब फौज के वीर जवानों के पास आधुनिक हथियार होते...,BJP,hi,NaN,True,65724
17403,1089768,1089768,sambitswaraj,जब फौज के वीर जवानों के पास आधुनिक हथियार होते...,BJP,hi,NaN,True,65724


In [6]:
meta = pd.read_csv("../lite_data/india_July_21.csv", header=0)
meta.head()

,Unnamed: 0,domain,id,id_str,party,screen_name,state
0,0,1.0,784325894559059968,784325894559059968,To Be Added,Sanjay_Meshack,To Be Added
1,1,1.0,741171197342187521,741171197342187520,BJP,BJP4AnN,Andaman and Nicobar Islands
2,2,1.0,1283244990274002949,1283244990274002944,INC,NSubramanian1,To Be Added
3,3,1.0,935363557050298368,935363557050298368,INC,SatishYepuri,Andhra Pradesh
4,4,1.0,112384119,112384119,BJP,praveenkubjp,Bihar


In [7]:
screen2state = meta.set_index('screen_name').to_dict()['state']
screen2party = dup_data.set_index('screen_name').to_dict()['party']

In [8]:
connections = dup_data.groupby('text_id')['screen_name'].apply(list).reset_index()
connections.head()

,text_id,screen_name
0,0,"[sachinkhurai, BjpYova_Rahul]"
1,1,"[jivaramchoudhar, VasundharaBJP]"
2,2,"[Johnsondodemani, Ansar_Salmara, syedmusyeeb, ..."
3,3,"[SanjayS62023183, KrishanKantJha8]"
4,4,"[netikchoudhary, NSUIMumbai]"


In [9]:
nodes = defaultdict(int)
edges = defaultdict(int)

In [10]:
for idx, row in connections.iterrows():
    screenlist = row['screen_name']
    for i in range(len(screenlist)):
        nodes[screenlist[i]]+=1
        
    for i in range(len(screenlist)-1):
        for j in range(i+1,len(screenlist)):
            stuple = sorted([screenlist[i], screenlist[j]])
            edges[(stuple[0], stuple[1])]+=1

In [11]:
edges

defaultdict(int,
            {('BjpYova_Rahul', 'sachinkhurai'): 1,
             ('VasundharaBJP', 'jivaramchoudhar'): 3,
             ('Ansar_Salmara', 'Johnsondodemani'): 90,
             ('Johnsondodemani', 'syedmusyeeb'): 143,
             ('Johnsondodemani', 'gouda_basan'): 1,
             ('IycMedia', 'Johnsondodemani'): 172,
             ('Ansar_Salmara', 'syedmusyeeb'): 85,
             ('Ansar_Salmara', 'gouda_basan'): 1,
             ('Ansar_Salmara', 'IycMedia'): 92,
             ('gouda_basan', 'syedmusyeeb'): 4,
             ('IycMedia', 'syedmusyeeb'): 632,
             ('IycMedia', 'gouda_basan'): 4,
             ('KrishanKantJha8', 'SanjayS62023183'): 1,
             ('NSUIMumbai', 'netikchoudhary'): 71,
             ('SachinPilot', 'minnatrahmani'): 2,
             ('HarshuBhaiyya', 'MLARamkhelavan'): 1,
             ('anupamanbsr', 'madhusharmabjp'): 1,
             ('DrPremKrBihar', 'anpandeybjp'): 1,
             ('kuldipdodiya781', 'shahritesh07'): 2,
             

In [16]:
import networkx as nx

In [17]:
G = nx.Graph()

In [18]:
for e in edges:
    if edges[e] > 10:
        if e[0] not in G.nodes:
            G.add_node(e[0], size=nodes[e[0]], party=screen2party[e[0]], state=screen2state[e[0]])
        if e[1] not in G.nodes:
            G.add_node(e[1], size=nodes[e[1]], party=screen2party[e[1]], state=screen2state[e[1]])
            
        G.add_edge(e[0], e[1], weight=edges[e])
        

In [19]:
nx.write_gexf(G, "../lite_data/astroturf_sametweets_shorter.gexf", encoding='utf-8')